# Unsupervised Learning Predict Student Solution

© Explore Data Science Academy

---
### Honour Code

I {**Kobus Le Roux, Devon Woodman, Nhlanhla Mthembu, Koketso Maraba, Tebogo Khoza, Mxolisi Zulu, Cara Brits**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Predict Overview: Movie Recommendation

__UPDATE ME__

### Problem Statement

__UPDATE ME__

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Data Engineering</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>

<a href=#seven>7. Conclusions</a>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>


Let's start by turning off unnecessary warnings in the code, to improve the aesthetics of our notebook:

In [349]:
# Turning off unnecessary warnings
import warnings
warnings.filterwarnings('ignore')

First, we include a list of uncommon packages that may need to be installed on your system. Uncomment and run the following if you need some of the packages listed:

In [350]:
# !pip install wordcloud
# !pip install contractions
# !pip install catboost
# !pip install xgboost
# !pip install comet_ml
# !pip install emoji


Next, we import all necessary modules:

In [351]:
# Libraries for Data Loading and Manipulation
import pandas as pd  # Pandas provides flexible data structures to manipulate structured data.
import numpy as np  # Numpy offers powerful data structures and tools for numerical computation.

# Text Processing Libraries
import contractions  # Contractions is used to handle English contractions, converting them into their longer forms.
import emoji  # Emoji allows easy manipulation and analysis of emojis in the text.
from nltk.corpus import stopwords  # Stopwords module provides a list of common words to be removed from the text.
from nltk.stem import WordNetLemmatizer  # WordNetLemmatizer is used for lemmatizing words, bringing them to their root form.
from nltk import download as nltk_download  # For downloading nltk packages, here 'wordnet'.
import regex  # Regex is used for regular expression matching and manipulation.
import string  # Provides constants and classes for string manipulation.
import re # Provides common string operations
import unicodedata  # Provides access to the Unicode Character Database for processing Unicode characters.

#Data monitoring Libraries:
from comet_ml import Experiment  # Allows developers to track, compare, explain and optimize experiments and models.

# Machine Learning Libraries
from catboost import CatBoostClassifier  # High-performance gradient boosting on decision trees library.
from scipy.sparse import hstack  # Used for stacking sparse matrices horizontally.
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer  # Transforms text into feature vectors for machine learning.
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix  # Metrics for evaluating machine learning models.
from sklearn.model_selection import train_test_split, GridSearchCV  # Provides utilities to split data and tune hyperparameters.
from sklearn.naive_bayes import GaussianNB, MultinomialNB  # Naive Bayes classifiers for machine learning.
from sklearn.linear_model import LogisticRegression # Logistic Regression classifier for machine learning.
from sklearn.tree import DecisionTreeClassifier # Decision Tree classifier for machine learning.
from sklearn.ensemble import RandomForestClassifier # Random Forrest classifier for machine learning
from sklearn.svm import LinearSVC, SVC  # Support Vector Machine classifiers.
from xgboost import XGBClassifier  # XGBoost is an efficient and flexible gradient boosting library.
import tensorflow as tf # For use in Neural Net
from tensorflow.keras.layers import Dense # For building neural net 
from tensorflow.keras.models import Sequential # For building neural net 
from tensorflow.keras import utils, backend as K # For neural net add ons 

# Feature selection Libraries:
from sklearn.feature_selection import SelectKBest # To reduce features
from sklearn.feature_selection import chi2 # Used to estimate which features is most impactful

# Data Visualization Libraries
import matplotlib.pyplot as plt  # Provides a way to visualize data (plotting, charts, etc.).
import seaborn as sns  # Provides a high-level interface for attractive, informative statistical graphics.
from sklearn.metrics import ConfusionMatrixDisplay # To visualize confusion matirces
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator  # Wordcloud is used to visualize word frequency in the text.
sns.set_theme(style="whitegrid")  # Sets the style for seaborn plots.

# Utility Libraries
import pickle # For saving models to file
from sklearn.utils import resample # For resampling unbalanced data
from imblearn.combine import SMOTETomek


# Downloading necessary NLTK corpus
nltk_download('wordnet')

# Global Constants for reproducibility and consistency
TRAIN_TEST_SPLIT_VAR = 0.2
RAND_STATE = 42
MAX_TEXT_FEATURES = 50000 # For all algorithms with exception to those listed below
RAD_SVC_Text_Features = 3000 # Specifically for use in radial svc
NN_Text_Features = 5000 #Specifically for use of neural net
XGB_Text_Features = 3000 # Specifically for use in XGBoost algorithm
RESAMPLE_ENTRY_AMOUNT = 4500 # If resampling, how many entries per category (Between 1000 and 6000)
VEC_MIN_WORD_TO_REMOVE = 1 # Remove words that occurs less that this value in dataset 


# Flags for notebook Execution
COMET_FLAG = False # To gauge wether to commit experiments to Comet ML
VECTORIZER_TO_USE = "tfidf" # Chooses between TfIDF vectorizer or count Vectorizer - accepted values are "tfidf" or "count"
RESAMPLE = False # Whether to resample training data
REMOVE_OVERUSED_WORDS = False # Wether to remove non-class specific words that occurs a lot

# Comet variables for logging experiments
COMET_API_KEY = "###########"
COMET_PROJECT_NAME = "########"
COMET_WORKSPACE = "#########"


# Select which models to tune hyperparameters on
HYPERPARAM_TO_Tune = {
    "Logistic Regression" : False,
    "Decision Tree": False,
    "Random Forrest": False,
    "Linear Support Vector": False,
    "Radial Support Vector": False,
    "Gaussian Naive Bayes": False,
    "Multinomial Naive Bayes": False,
    "CatBoost": False,
    "XGBoost": False
}





[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kobus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

We load the data from the datafiles:

<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

Before we can jump in and construct models for the data, it is important to investigate and understand the features present within the dataset


<a id="four"></a>
## 4. Data Engineering and NLP Preprocessing
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>



To capture all the preprocessing happening within the data, we start a comet experiment:

In [384]:
if COMET_FLAG:
  # Start comet experiment:
  experiment = Experiment(
    api_key = COMET_API_KEY,
    project_name = COMET_PROJECT_NAME,
    workspace= COMET_WORKSPACE
  )

  experiment.add_tag("Data Preprocessing")
  experiment.set_name("Data Preprocessing")

We first begin by reloading the data and discarding the changes made during the EDA section:

Ending the comet experiment for data preprocessing:

In [414]:
if COMET_FLAG:
    experiment.end()

<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>



### 5.1 Logistic regression

Logistic regression is a classification algorithm used to predict the probability of a binary outcome based on one or more input features. It models the relationship between the input variables and the probability of the outcome belonging to a particular class. Logistic regression uses the logistic function (also known as the sigmoid function) to map the output of a linear combination of the input features to a value between 0 and 1, representing the probability of belonging to the positive class.
In simpler terms, logistic regression aims to find the best-fitting S-shaped curve that separates the two classes. It estimates the coefficients (weights) of the input features through a process called maximum likelihood estimation, optimizing the parameters to maximize the likelihood of the observed data.

Once trained, logistic regression can make predictions by calculating the probability of the positive class based on the input features. A threshold is then applied to determine the final predicted class.

Logistic regression models are known for their simplicity and interpetability. Since they are more simplistic models, they are relatively quick to train and computationally efficient. They can also be expanded to handle multiclass classification as is the case for our data. This model does assume a linear relationship between the features and the log-odds of the outcome, however, which does not necessarily hold true in many cases. It is also sensitive to outliers and irrelevant features. 

Lets begin our training process, by starting our Comet ML experiment, so that all aspects of the model gets logged for comparison:

In [415]:
if COMET_FLAG:
  # Start comet experiment:
  experiment = Experiment(
    api_key = COMET_API_KEY,
    project_name = COMET_PROJECT_NAME,
    workspace= COMET_WORKSPACE
  )

  experiment.add_tag("LogisticRegression")
  experiment.set_name("LogisticRegression")

else:
  print("Experiment not logged")

Experiment not logged


We begin by initializing a Logistic Regression classifier , and fitting it to the data:


In [416]:
# Creating a Logistic Regression Classifier object:
log_reg = LogisticRegression()

# Fitting the model to our training data subset
log_reg.fit(X_train, y_train)

LogisticRegression()

To get a sense of how well this model is performing, let's test the model on the testing subset of our training data:


In [417]:
# Predict values of the testing subset
pred_log_reg = log_reg.predict(X_test)

# Let's produce a classification report of the model as is:
print(classification_report(y_test, pred_log_reg))

              precision    recall  f1-score   support

        -1.0       0.82      0.26      0.39       278
         0.0       0.62      0.38      0.47       425
         1.0       0.73      0.90      0.81      1755
         2.0       0.78      0.74      0.76       706

    accuracy                           0.73      3164
   macro avg       0.74      0.57      0.61      3164
weighted avg       0.73      0.73      0.71      3164



Form the classification report, we see that our recall is relatively low for our two least represented categories, namely `Anti climate change` and `Neutral` sentiments. Our remaining sentiments scores pretty high in all the metrics we observe. This model performs pretty well in overall predictive performance.


If interested, one can try to gauge the effects of hyperparameter tuning on the model performance. Note: For the purposes of this notebook, hyperparameter tuning will be ignored since it can be very timely to run.

In [418]:
if HYPERPARAM_TO_Tune["Logistic Regression"]:
    # Set hyper-parameters to tune 
    log_reg_hyperparameters =     {
    'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 5),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500]
    } 

    # Search over hyperparameter grid
    log_reg_gridsearch = GridSearchCV(LogisticRegression(), log_reg_hyperparameters, scoring='f1_weighted', cv =3, n_jobs=3, verbose=3) 

    # Fit grid of models to the data
    log_reg_gridsearch.fit(X_train, y_train)
    best_params = log_reg_gridsearch.best_params_
else:
    log_reg_hyperparameters = {
    'penalty' : "default",
    'C' : "default",
    'solver' : "default",
    'max_iter' : "default"
    } 

    best_params = {
    'penalty' : 'l2',
    'C' : 1.0,
    'solver' : 'lbfgs',
    'max_iter' : 100
    } 

Let us save our metrics for our model considered, in order to log it to CometML later


In [419]:
if HYPERPARAM_TO_Tune["Logistic Regression"]:
    # Get best predictor:
    best_log_reg = log_reg_gridsearch.best_estimator_

    # Use best estimator to make predictions
    pred_best_log_reg = best_log_reg.predict(X_test)

    # Print optimised classification report:
    print(classification_report(y_test, pred_best_log_reg))

    # Produce confusion matrix, f1 score, precision and recall for model in order to log them to comet
    log_reg_cm = confusion_matrix(y_test, pred_best_log_reg)
    log_reg_f1 = f1_score(y_test, pred_best_log_reg, average="weighted") # average only necessary if model nly deals in binary classification
    log_reg_precision = precision_score(y_test, pred_best_log_reg, average='weighted')
    log_reg_recall = recall_score(y_test, pred_best_log_reg, average="weighted")

else:
    log_reg_cm = confusion_matrix(y_test, pred_log_reg)
    log_reg_f1 = f1_score(y_test, pred_log_reg, average="weighted") # average only necessary if model nly deals in binary classification
    log_reg_precision = precision_score(y_test, pred_log_reg, average='weighted')
    log_reg_recall = recall_score(y_test, pred_log_reg, average="weighted")
    


Next, we log the metrics calculated above to use within comet:


In [420]:
# Create logging info

# Add all relevant info pertaining to your model here
log_reg_params = {"random_state": RAND_STATE, # DO not remove this one
          "model_type": "log_reg",
          "scaler": "no scaling", # DO not remove this one
          "param_grid": str(log_reg_hyperparameters),
          "best_param": str(best_params),
          "text_feature_count": str(MAX_TEXT_FEATURES), # DO not remove this one
          "vectorizer": VECTORIZER_TO_USE, # DO not remove this one
          "train_test_split": str(TRAIN_TEST_SPLIT_VAR), # DO not remove this one
          "Resample": str(RESAMPLE)
          }
log_reg_metrics = {"f1": log_reg_f1,
           "recall": log_reg_recall,
           "precision": log_reg_precision,
           "confusion_matrix": log_reg_cm
           }

if COMET_FLAG:
    # Log all important info
    experiment.log_parameters(log_reg_params)
    experiment.log_metrics(log_reg_metrics)
    experiment.log_confusion_matrix(matrix=log_reg_cm)

In order to produce a submission to Kaggle, we retrain the model (using the optimised hyperparameters if applicable) on the complete set of training data:


In [421]:
# Initiate final model
final_log_reg = LogisticRegression( **best_params) # Use determined hyperparameters here

# Train on all available data
final_log_reg.fit(x_train_dataset, y_train_dataset)

# Generate predictions for the evaluation dataset
log_reg_kaggle_predictions = final_log_reg.predict(x_evaluate_dataset)


Finally, we produce a csv of the predictions to submit to Kaggle


In [422]:
# Produce kaggle submission csv 

# Make submissions into dataframe
df_log_reg_kaggle_predictions = pd.DataFrame(log_reg_kaggle_predictions, columns=["sentiment"])

# Convert sentiment to integer
df_log_reg_kaggle_predictions['sentiment'] = df_log_reg_kaggle_predictions['sentiment'].astype('int')

# Match submissions to tweet id's
df_log_reg_kaggle_submission = pd.concat([df_tweetid_for_submission.reset_index(drop=True), df_log_reg_kaggle_predictions.reset_index(drop=True)], axis=1,)


#Save submission as csv
df_log_reg_kaggle_submission.to_csv('LogisticRegressionPredictions.csv', index=False)

df_log_reg_kaggle_submission

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,1
...,...,...
10541,895714,1
10542,875167,1
10543,78329,2
10544,867455,0


Lastly, since we are done with the model, we remember to end the experiment and write logs to Comet ML

In [423]:
if COMET_FLAG:
    # Ending the experiment
    experiment.end()
else:
    print("Completed Logistic Regression: Experiment not logged")

Completed Logistic Regression: Experiment not logged


Saving the model for later use:

In [424]:
# Choose a filename to save model as
model_save_path = './Models/LogisticRegression.pkl'

# Save model to specified filename
with open(model_save_path, 'wb') as file:
    pickle.dump(final_log_reg,file)

<a id="six"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

To evaluate our model performance, we will make use of the `f1-score` as a metric...


<a id="seven"></a>
## 7. Conclusion
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>



---

In conclusion, our best performing model for predicting sentiment...

#### Future Considerations:


